# What does good AI middleware like Semantic Kernel do for your **Adaptability**? It gives gives you a good toolbelt to take on any situation that comes your way.

![](assets/toolbelt.png)

## 🔥 We warm up a kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: SkiaSharp, 2.88.3"

Installed Packages Microsoft.SemanticKernel, 1.1.0 SkiaSharp, 2.88.3 System.Linq.Async, 6.0.1

Loading extensions from `C:\Users\rnsmith\.nuget\packages\skiasharp\2.88.3\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [2]:
// Load settings
#!import config/Settings.cs 
#!import config/SkiaUtils.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

In [3]:
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using SemanticKernel = Microsoft.SemanticKernel.Kernel;

// Create a kernel with OpenAI chat completion
SemanticKernel kernel = SemanticKernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: model,
        apiKey: apiKey)
    .Build();

# 💬 How many ways can I prompt thee?

## ① We've got vanilla prompts easy-peasy for u

In [5]:
// Example 1. Invoke the kernel with a prompt and display the result
Console.WriteLine(await kernel.InvokePromptAsync("What is the favorite food of winnie the pooh"));
Console.WriteLine();

Error: Microsoft.SemanticKernel.HttpOperationException: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
Status: 429 (Too Many Requests)
ErrorCode: insufficient_quota

Content:
{
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}


Headers:
Date: Wed, 28 Feb 2024 22:14:36 GMT
Connection: keep-alive
Vary: REDACTED
X-Request-ID: REDACTED
Strict-Transport-Security: REDACTED
CF-Cache-Status: REDACTED
Set-Cookie: REDACTED
Server: cloudflare
CF-RAY: REDACTED
Alt-Svc: REDACTED
Content-Type: application/json; charset=utf-8
Content-Length: 337

 ---> Azur

## ② Can I use templates? We're getting more {{ handlebars-y }} ...

In [ ]:
// Example 2. Invoke the kernel with a templated prompt and display the result
KernelArguments arguments = new() { { "topic", "grand canyon" }, { "meal", "sushi" } };
Console.WriteLine(await kernel.InvokePromptAsync("What color is the {{$topic}} and how is {{$meal}} cooked?", arguments));
Console.WriteLine();

## ③ Got streaming? Yes, we have that in stock!

In [ ]:
await foreach (var update in kernel.InvokePromptStreamingAsync("What color is the {{$topic}}? Provide a detailed explanation.", arguments))
{
    Console.Write(update);
}
Console.WriteLine();

## ④ I need to tune my hyperparameters ... abs!

In [ ]:
arguments = new(new OpenAIPromptExecutionSettings { MaxTokens = 500, Temperature = 1.5 }) { { "topic", "dogs" } };
Console.WriteLine(await kernel.InvokePromptAsync("Tell me a story about {{$topic}}", arguments));

# 🌅 "I want images. Lots of them."

## ① 🖼️ You can auto-paint them with Dall-E

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.OpenAI;

using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0002, SKEXP0011, SKEXP0012

var builder = Kernel.CreateBuilder();

if(useAzureOpenAI)
{
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    builder.AddAzureOpenAITextToImage("dall-e-3",azureEndpoint, apiKey);
}
else
{
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);
    builder.AddOpenAITextToImage(apiKey, orgId);
}
   
var kernel = builder.Build();

// Get AI service instance used to generate images
var dallE = kernel.GetRequiredService<ITextToImageService>();


#pragma warning disable SKEXP0002

var prompt = @"
Think about an artificial object correlated to number {{$input}}.
Describe the image with one detailed sentence. The description cannot contain numbers.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 256,
    Temperature = 1
};

// Create a semantic function that generate a random image description.
var genImgDescription = kernel.CreateFunctionFromPrompt(prompt, executionSettings);

var random = new Random().Next(0, 200);
var imageDescriptionResult = await kernel.InvokeAsync(genImgDescription, new() { ["input"] = random });
var imageDescription = imageDescriptionResult.ToString();

// Use DALL-E 2 to generate an image. OpenAI in this case returns a URL (though you can ask to return a base64 image)
var imageUrl = await dallE.GenerateImageAsync(imageDescription.Trim(), 512, 512);

await SkiaUtils.ShowImage(imageUrl, 512, 512);

## ② 🦄 Or, you can read them with multimodal magic

In [ ]:
const string ImageUri = "https://i0.wp.com/maeda.pm/wp-content/uploads/2017/09/photo_dark_big.png";

await SkiaUtils.ShowImage(ImageUri, 512, 512);

# ⚠️ Note that you may not have access to `gpt-4-vision-preview` on OpenAI unless you have a paid subscription, and/or there's a waiting list to get access to it. Plus the name of the model is likely going to change. All model names for OAI are over [here](https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo).

In [ ]:
using System;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using SemanticKernel = Microsoft.SemanticKernel.Kernel;

var kernel = SemanticKernel.CreateBuilder()
    .AddOpenAIChatCompletion("gpt-4-vision-preview", apiKey)
    .Build();

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var chatHistory = new ChatHistory("You are a friendly assistant.");

chatHistory.AddUserMessage(new ChatMessageContentItemCollection
{
    new TextContent("What’s in this image?"),
    new ImageContent(new Uri(imageUrl))
    //new ImageContent(new Uri(ImageUri))
});

var reply = await chatCompletionService.GetChatMessageContentAsync(chatHistory);

Console.WriteLine(reply.Content);

# 🧲 And then there's the 🫲 left hand

In [ ]:
// TO BE CONTINUED ...